In [2]:
import pandas as pd

def target_encode(df, categorical_col, target_col, min_samples_leaf=1, smoothing=1):
    """
    Applies mean target encoding to a specified categorical column.

    Parameters:
        df (pd.DataFrame): The dataframe containing the data.
        categorical_col (str): The name of the categorical column to be encoded.
        target_col (str): The name of the target column.
        min_samples_leaf (int): Minimum samples to take category average into account.
        smoothing (float): Smoothing effect to balance category mean with global mean.

    Returns:
        pd.Series: The mean-encoded values for the specified categorical column.
    """
    # Compute the global mean of the target
    global_mean = df[target_col].mean()
    
    # Compute the number of samples and mean of each category
    agg = df.groupby(categorical_col)[target_col].agg(['mean', 'count'])
    means = agg['mean']
    counts = agg['count']
    
    # Compute the smoothed mean for each category
    smoothed_means = (counts * means + smoothing * global_mean) / (counts + smoothing)
    
    # Apply the smoothed mean to the original column
    encoded_col = df[categorical_col].map(smoothed_means)
    
    return encoded_col

# Example usage
data = pd.DataFrame({
    'City': ['New York', 'Los Angeles', 'Chicago', 'New York', 'Chicago', 'Los Angeles'],
    'Purchase': [1, 0, 1, 1, 0, 0]
})

data['City_encoded'] = target_encode(data, categorical_col='City', target_col='Purchase')
print(data)


          City  Purchase  City_encoded
0     New York         1      0.833333
1  Los Angeles         0      0.166667
2      Chicago         1      0.500000
3     New York         1      0.833333
4      Chicago         0      0.500000
5  Los Angeles         0      0.166667
